## Code#1 Information
<small> <quote>
**Author:**  Shafagh Keyvanian [shkey@seas.upenn.edu]  <br>
**Date**: *Spring 2024*
</quote> </small>

<Large> <quote>
**One Subjects**<br>
**Convert Motive export .csv to .npz Upper-body Euler angles**<br>
**Bones: Hip, Ab, Chest, UArm, FArm, Hand** </quote> </Large>

Input: OptiTrack data: Motive exported .csv file
- File: <small> <blockquote>    
        Folder: ../data/raw_exports/ <br>
        Name: subject#_Rh_clinical.csv  </blockquote> </small>
- Columns: <small> <blockquote> 
        Bones fields: rows=frames, col=(pos[x y z] quat(x y z w)) </blockquote> </small>
- Motive Properties <small> <blockquote>
        Skeleton: Conventional Upper (27 Markers) - 43 Bones<br>
        Gaps manually filled with "Linear Interpolation"<br>
        Export: Bones- position & quaternion, Markers- position </blockquote> </small>

Output: 
.npz file containing Upper-body Euler angles
- File: <small> <blockquote> 
        Folder: ../data/prossesed_exports/takes <br>
        Name: subject#_Rh_clinical_7dof_eul_angles.csv </blockquote> </small>
- Columns: <small> <blockquote> 
        'time', 'eul_right', 'eul_left', 'eul_head' </blockquote> </small>
- Chains: <small> <blockquote>
        chain_R = ['Skeleton', 'Ab', 'Chest', 'RUArm','RFArm','RHand']<br>
        chain_L = ['Skeleton', 'Ab', 'Chest', 'LUArm','LFArm','LHand']<br>
        chain_H = ['Skeleton', 'Ab', 'Chest', 'Neck', 'Head'] </blockquote> </small>

In [ ]:
#%% import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
from tf.transformations import quaternion_matrix, euler_from_matrix, euler_matrix

In [ ]:
#%% Read CSV file
csvfile = '../data/raw_exports/subject0_Rh_clinical.csv' 
raw_data = pd.read_csv(csvfile, header=[3, 5, 6], index_col=0)
raw_data.rename(columns=lambda x: x.replace('Skeleton:', ''), level=0, inplace=True)
time = pd.read_csv(csvfile, header=6, usecols=[1]).values
# print(raw_data.head())

chain_R = ['Skeleton', 'Ab', 'Chest', 'RUArm', 'RFArm', 'RHand']
chain_L = ['Skeleton', 'Ab', 'Chest', 'LUArm', 'LFArm', 'LHand']
chain_H = ['Skeleton', 'Ab', 'Chest', 'Neck', 'Head']

skl_R = raw_data[chain_R].copy()
skl_L = raw_data[chain_L].copy()
skl_H = raw_data[chain_H].copy()

# 7 DoF arm: change skl['Chest'] with position=mean(RShoulder, LShoulder) 
skl_R[('Chest', 'Position')] = (raw_data[('RShoulder', 'Position')] + raw_data[('LShoulder', 'Position')])/2
skl_L[('Chest', 'Position')] = (raw_data[('RShoulder', 'Position')] + raw_data[('LShoulder', 'Position')])/2
skl_H[('Chest', 'Position')] = (raw_data[('RShoulder', 'Position')] + raw_data[('LShoulder', 'Position')])/2

In [ ]:
#%% Visualization of joint locations at time t
t = 1000
eul_seq = 'rzxy'

# Offset angle between strnm_dir_R/L and strnm_dir_H
strnm_dir_R = raw_data[('RShoulder', 'Position')] - raw_data[('Chest', 'Position')]
strnm_dir_L = raw_data[('LShoulder', 'Position')] - raw_data[('Chest', 'Position')]
strnm_dir_H = raw_data[('Neck', 'Position')] - raw_data[('Chest', 'Position')]
strnm_ang_R = np.arccos(np.dot(strnm_dir_R.iloc[t], strnm_dir_H.iloc[t])/(np.linalg.norm(strnm_dir_R.iloc[t])*np.linalg.norm(strnm_dir_H.iloc[t])))
strnm_ang_L = np.arccos(np.dot(strnm_dir_L.iloc[t], strnm_dir_H.iloc[t])/(np.linalg.norm(strnm_dir_L.iloc[t])*np.linalg.norm(strnm_dir_H.iloc[t])))


def get_positions(skl, chain):  
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
    return positions - positions[0]

def get_euler(skl, chain):  
    eul = []
    R_prox = np.eye(3)
    for j in range(len(chain)):
        q = skl[(chain[j], 'Rotation')].iloc[t]
        R_global = quaternion_matrix(q)[:3,:3]
        R_rel = R_prox.T @ R_global
        R_prox = R_global
        eul_values = euler_from_matrix(R_rel, axes=eul_seq)
        # (ang1, ang2, ang3) = euler_from_matrix(R_rel, axes=eul_seq)
        # eul_values = np.array([ang1, ang2, ang3])
        eul.append(eul_values)
    return np.array(eul)

def forward_kinematics(skl, chain): 
    bone_pos = get_positions(skl, chain)
    bone_length = np.linalg.norm(np.diff(bone_pos, axis=0), axis=1)
    eul_angles = get_euler(skl, chain)
    
    fk_pose = np.zeros((len(eul_angles), 3))
    pos_prox = np.zeros(3)
    R_prox = np.eye(3)
    fk_pose[0] = bone_pos[0]
    for i in range(2):
        R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
        R_prox = R_prox @ R_rel
        pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox

    for i in range(2, len(eul_angles)-1):
        R_rel = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
        R_prox = R_prox @ R_rel
        if chain==chain_R:  
            if i==2:
                R_off = euler_matrix(0, 0, strnm_ang_R, axes=eul_seq)[:3, :3]
                pos_prox += R_prox @ R_off @ np.array([bone_length[i], 0, 0])
            else:
                pos_prox += R_prox @ np.array([bone_length[i], 0, 0])
        elif chain==chain_L:
            if i==2:
                R_off = euler_matrix(0, 0, -strnm_ang_L, axes=eul_seq)[:3, :3]
                pos_prox += R_prox @ R_off @ np.array([-bone_length[i], 0, 0])
            else:
                pos_prox += R_prox @ np.array([-bone_length[i], 0, 0])
        elif chain==chain_H:
            pos_prox += R_prox @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = pos_prox
    return fk_pose

# Get bone positions from raw data
pos_raw_R = get_positions(skl_R, chain_R)
pos_raw_L = get_positions(skl_L, chain_L)
pos_raw_H = get_positions(skl_H, chain_H)

# Get bone positions using forward kinematics
pos_fk_R = forward_kinematics(skl_R, chain_R)
pos_fk_L = forward_kinematics(skl_L, chain_L)
pos_fk_H = forward_kinematics(skl_H, chain_H)

eul_angles_R = get_euler(skl_R, chain_R)
eul_angles_L = get_euler(skl_L, chain_L)
eul_angles_H = get_euler(skl_H, chain_H)

# print("{:<10} {:<25} {:<25} {:<25}".format("pos_R:", "Raw", "FK", "Euler Angles"))
print("{:<10} {:<25} {:<25} {:<25}".format("Position", "Raw", "FK", "Error"))
for j in range(len(chain_H)):
    print("{:<10} {:<25} {:<25} {:<25}".format(chain_H[j], str(np.around(pos_raw_H[j], 2)), str(np.around(pos_fk_H[j], 2)), str(np.around(pos_fk_H[j]-pos_raw_H[j], 2))))
for j in range(3, len(chain_R)):
    print("{:<10} {:<25} {:<25} {:<25}".format(chain_R[j], str(np.around(pos_raw_R[j], 2)), str(np.around(pos_fk_R[j], 2)), str(np.around(pos_fk_R[j]-pos_raw_R[j], 2))))
for j in range(3, len(chain_L)):
    print("{:<10} {:<25} {:<25} {:<25}".format(chain_L[j], str(np.around(pos_raw_L[j], 2)), str(np.around(pos_fk_L[j], 2)), str(np.around(pos_fk_L[j]-pos_raw_L[j], 2))))        

print("\n{:<15} {:<10} {:<10} {:<10}".format("Euler Angles", "0", "1", "2"))
for j in range(len(chain_H)):
    print("{:<15} {:<10} {:<10} {:<10}".format(chain_H[j], str(np.around(eul_angles_H[j,0]*180/np.pi, 2)), str(np.around(eul_angles_H[j,1]*180/np.pi, 2)), str(np.around(eul_angles_H[j,2]*180/np.pi, 2))))   
for j in range(3, len(chain_R)):
    print("{:<15} {:<10} {:<10} {:<10}".format(chain_R[j], str(np.around(eul_angles_R[j,0]*180/np.pi, 2)), str(np.around(eul_angles_R[j,1]*180/np.pi, 2)), str(np.around(eul_angles_R[j,2]*180/np.pi, 2))))
for j in range(3, len(chain_L)):
    print("{:<15} {:<10} {:<10} {:<10}".format(chain_L[j], str(np.around(eul_angles_L[j,0]*180/np.pi, 2)), str(np.around(eul_angles_L[j,1]*180/np.pi, 2)), str(np.around(eul_angles_L[j,2]*180/np.pi, 2))))

# Plot positions using raw position data and forward kinematics
fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(11, 4.5), constrained_layout=True, num='Joint Locations')
# fig.subplots_adjust(left=-0.5, wspace=0.08, right=1.2)
for ax in axes:
    ax.set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
    ax.view_init(elev=25, azim=150)
ax1, ax2 = axes[0], axes[1]
ax1.set_title('Joint Locations Using Position Data', pad=8)
ax2.set_title('Joint Locations Using Forward Kinematics', pad=8)
# colors = ['k', 'y', 'm', 'c', 'r', 'g', 'b']
colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
for j in range(len(chain_R)):
    ax1.scatter(pos_raw_R[j][0], pos_raw_R[j][1], pos_raw_R[j][2], label=f"{chain_R[j]}, Raw", marker='*', s=50, color=colors_R[j])
    ax1.scatter(pos_raw_L[j][0], pos_raw_L[j][1], pos_raw_L[j][2], label=f"{chain_L[j]}, Raw", marker='*', s=50, color=colors_L[j])
    ax2.scatter(pos_fk_R[j][0], pos_fk_R[j][1], pos_fk_R[j][2], label=f"{chain_R[j]}, FK", marker='o', s=50, color=colors_R[j])
    ax2.scatter(pos_fk_L[j][0], pos_fk_L[j][1], pos_fk_L[j][2], label=f"{chain_L[j]}, FK", marker='o', s=50, color=colors_L[j])
    if j!=0:
        ax1.plot([pos_raw_R[j][0], pos_raw_R[j-1][0]], [pos_raw_R[j][1], pos_raw_R[j-1][1]], [pos_raw_R[j][2], pos_raw_R[j-1][2]], 'k')
        ax1.plot([pos_raw_L[j][0], pos_raw_L[j-1][0]], [pos_raw_L[j][1], pos_raw_L[j-1][1]], [pos_raw_L[j][2], pos_raw_L[j-1][2]], 'b')
        ax2.plot([pos_fk_R[j][0], pos_fk_R[j-1][0]], [pos_fk_R[j][1], pos_fk_R[j-1][1]], [pos_fk_R[j][2], pos_fk_R[j-1][2]], 'k')
        ax2.plot([pos_fk_L[j][0], pos_fk_L[j-1][0]], [pos_fk_L[j][1], pos_fk_L[j-1][1]], [pos_fk_L[j][2], pos_fk_L[j-1][2]], 'b')
for j in range(1, len(chain_H)):
    ax1.scatter(pos_raw_H[j][0], pos_raw_H[j][1], pos_raw_H[j][2], label=f"{chain_H[j]}, Raw", marker='*', s=50, color='black')
    ax2.scatter(pos_fk_H[j][0], pos_fk_H[j][1], pos_fk_H[j][2], label=f"{chain_H[j]}, FK", marker='o', s=50, color='black')
    if j!=0:
        ax1.plot([pos_raw_H[j][0], pos_raw_H[j-1][0]], [pos_raw_H[j][1], pos_raw_H[j-1][1]], [pos_raw_H[j][2], pos_raw_H[j-1][2]], 'k--')
        ax2.plot([pos_fk_H[j][0], pos_fk_H[j-1][0]], [pos_fk_H[j][1], pos_fk_H[j-1][1]], [pos_fk_H[j][2], pos_fk_H[j-1][2]], 'k--')
ax2.legend(handles=
    [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
    [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
    [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))
plt.show()


In [ ]:
#%% Store Euler angles

# Solve Right chain Euler angles
eul_seq = 'rzxy'
eul_right = np.zeros((len(time), len(chain_R), 3))
for t in range(len(time)):
    eul = []
    R_prox = np.eye(3)
    for j in range(len(chain_R)):
        q = skl_R[(chain_R[j], 'Rotation')].iloc[t]
        R_global = quaternion_matrix(q)[:3,:3]
        R_rel = R_prox.T @ R_global
        R_prox = R_global
        eul_values = euler_from_matrix(R_rel, axes=eul_seq)
        eul.append(eul_values)
    eul_right[t] = np.array(eul)

# Solve Left chain Euler angles
eul_left = np.zeros((len(time), len(chain_L), 3))
for t in range(len(time)):
    eul = []
    R_prox = np.eye(3)
    for j in range(len(chain_L)):
        q = skl_L[(chain_L[j], 'Rotation')].iloc[t]
        R_global = quaternion_matrix(q)[:3,:3]
        R_rel = R_prox.T @ R_global
        R_prox = R_global
        eul_values = euler_from_matrix(R_rel, axes=eul_seq)
        eul.append(eul_values)
    eul_left[t] = np.array(eul)

# # Solve Head chain Euler angles
# eul_head = np.zeros((len(time), len(chain_H), 3))
# for t in range(len(time)):
#     eul = []
#     R_prox = np.eye(3)
#     for j in range(len(chain_H)):
#         q = skl_H[(chain_H[j], 'Rotation')].iloc[t]
#         R_global = quaternion_matrix(q)[:3,:3]
#         R_rel = R_prox.T @ R_global
#         R_prox = R_global
#         eul_values = euler_from_matrix(R_rel, axes=eul_seq)
#         eul.append(eul_values)
#     eul_head[t] = np.array(eul)

# # Save Euler angles to npz file with automatic naming
# npz_file = '../data/processed_exports/takes' + csvfile.split('/').split('_')[0] + '_7dof_eul_angles.npz'
# np.savez(npz_file, time=time, eul_right=eul_right, eul_left=eul_left, eul_head=eul_head)

In [ ]:
#%% Data visualization
# Initialize variables
n_dof = 8    
dof_names = ['Shoulder Abduction', 'Shoulder Rotation', 'Shoulder Extension', 'Elbow Flexion', \
             'Elbow Supination', 'Wrist Deviation', 'Wrist Supination', 'Wrist Flexion']
dof_codes = ['sh_abd', 'sh_rot', 'sh_ext', 'el_fle', 'el_sup', 'wr_dev', 'wr_sup', 'wr_fle']

time_ed = time
eul_right_ed = eul_right
eul_left_ed = eul_left

# Remove frames with specific indices, add 5 frames instead of the removed frames, linearly interpolated
def remove_frames(data, from_ind, to_ind):
    diff = data[to_ind]-data[from_ind] 
    print("data[from_ind]: ", data[from_ind])
    print("diff: ", diff    )
    data = np.delete(data, range(from_ind, to_ind), axis=0)
    # if len(data.shape) == 3:
    #     for i in range(5):
    #         data = np.insert(data[from_ind], from_ind+i, (data[from_ind] + diff*(i+1)/6).reshape(1, data.shape[1], data.shape[2]), axis=0)
    #     return data
    # else:       
    for i in range(5):
        data = np.insert(data, from_ind+i, (data[from_ind] + diff*(i+1)/6), axis=0)
    return data

time_ed = remove_frames(time, 14000, 20000)
eul_right_ed = remove_frames(eul_right_ed, 14000, 20000)
eul_left_ed = remove_frames(eul_left_ed, 14000, 20000)

#%% Arm angles
R_sh_abd = -np.around(eul_right_ed[:,3,0] * 180/np.pi, 2)
R_sh_rot =  np.around(eul_right_ed[:,3,1] * 180/np.pi, 2)
R_sh_ext = -np.around(eul_right_ed[:,3,2] * 180/np.pi, 2)
R_el_fle =  np.around(eul_right_ed[:,4,0] * 180/np.pi, 2)
R_el_sup =  np.around(eul_right_ed[:,4,1] * 180/np.pi, 2)
R_wr_dev = -np.around(eul_right_ed[:,5,0] * 180/np.pi, 2)
R_wr_sup =  np.around(eul_right_ed[:,5,1] * 180/np.pi, 2)
R_wr_fle = -np.around(eul_right_ed[:,5,2] * 180/np.pi, 2)

L_sh_abd =  np.around(eul_left_ed[:,3,0] * 180/np.pi, 2)
L_sh_rot =  np.around(eul_left_ed[:,3,1] * 180/np.pi, 2)
L_sh_ext =  np.around(eul_left_ed[:,3,2] * 180/np.pi, 2)
L_el_fle = -np.around(eul_left_ed[:,4,0] * 180/np.pi, 2)
L_el_sup =  np.around(eul_left_ed[:,4,1] * 180/np.pi, 2)
L_wr_dev =  np.around(eul_left_ed[:,5,0] * 180/np.pi, 2)
L_wr_sup =  np.around(eul_left_ed[:,5,1] * 180/np.pi, 2)
L_wr_fle =  np.around(eul_left_ed[:,5,2] * 180/np.pi, 2)

# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("DoF", "L_min", "L_max", "R_min", "R_max"))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Shoulder Rot", str(min(L_sh_rot)), str(max(L_sh_rot)), str(min(R_sh_rot)), str(max(R_sh_rot))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Shoulder Abd", str(min(L_sh_abd)), str(max(L_sh_abd)), str(min(R_sh_abd)), str(max(R_sh_abd))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Shoulder Ext", str(min(L_sh_ext)), str(max(L_sh_ext)), str(min(R_sh_ext)), str(max(R_sh_ext))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Elbow Fle", str(min(L_el_fle)), str(max(L_el_fle)), str(min(R_el_fle)), str(max(R_el_fle))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Elbow Sup", str(min(L_el_sup)), str(max(L_el_sup)), str(min(R_el_sup)), str(max(R_el_sup))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Wrist Fle", str(min(L_wr_fle)), str(max(L_wr_fle)), str(min(R_wr_fle)), str(max(R_wr_fle))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Wrist Dev", str(min(L_wr_dev)), str(max(L_wr_dev)), str(min(R_wr_dev)), str(max(R_wr_dev))))
# print("{:<20} {:<10} {:<15} {:<10} {:<15}".format("Wrist Sup", str(min(L_wr_sup)), str(max(L_wr_sup)), str(min(R_wr_sup)), str(max(R_wr_sup))))

#%% Plot right arm data vs time for diagnostics
i_start = 14000 
i_end = 20000 #len(time) 
fig0, ax0 = plt.subplots(1, 1, sharex=False, figsize=(12, 6))
if i_start < len(time):
    for d in range(n_dof):
        ax0.plot(time[i_start:i_end], eval(f"R_{dof_codes[d]}")[i_start:i_end], label=f'{dof_names[d]}')
        ax0.set(xlabel='Time (s)', ylabel='Right Arm Angle (deg)', xlim=(time[i_start], time[i_end]+15))
    ax0.legend()
    ax0.grid()
    # ax0.xaxis.set_major_locator(plt.MultipleLocator(2))

#%% Plot left arm data vs time for diagnostics
i_start = 12000 
i_end = 22000 #len(time) 
fig0, ax0 = plt.subplots(1, 1, sharex=False, figsize=(12, 6))
if i_start < len(time):
    for d in range(n_dof):
        ax0.plot(time[i_start:i_end], eval(f"L_{dof_codes[d]}")[i_start:i_end], label=f'{dof_names[d]}')
        ax0.set(xlabel='Time (s)', ylabel='Left Arm Angles (deg)', xlim=(time[i_start], time[i_end]+15))
    ax0.legend()
    ax0.grid()

plt.show()

In [ ]:
# Draw Euler angles for right chain
fig, axes = plt.subplots(len(chain_R), 3, figsize=(8, 16), constrained_layout=True, num='right_chain_angles')
fig.suptitle('Right Chain Euler Angles', fontsize=14)
for j in range(len(chain_R)):
    eul_to_draw = eul_right[:, j] * 180/np.pi
    # round to 2 decimal points
    eul_to_draw = np.around(eul_to_draw, 2)
    # axes[j,0].scatter(eul_to_draw[:len(time_cl),0], eul_to_draw[:len(time_cl),1], label=f"{chain_R[j]}, 0, 1", marker='o', s=20, color='black')
    # axes[j,0].scatter(eul_to_draw[len(time_cl):,0], eul_to_draw[len(time_cl):,1], label=f"{chain_R[j]}, 0, 1", marker='o', s=20, color='blue')
    axes[j,0].scatter(eul_to_draw[:,0], eul_to_draw[:,1], label=f"{chain_R[j]}, 0, 1", marker='o', s=20, color='black')
    axes[j,1].scatter(eul_to_draw[:,1], eul_to_draw[:,2], label=f"{chain_R[j]}, 1, 2", marker='o', s=20, color='black')
    axes[j,2].scatter(eul_to_draw[:,2], eul_to_draw[:,0], label=f"{chain_R[j]}, 2, 0", marker='o', s=20, color='black')
    axes[j,0].set(ylabel=(f"{chain_R[j]}, ax 0 vs 1"))
    axes[j,1].set(ylabel=(f"{chain_R[j]}, ax 1 vs 2"))
    axes[j,2].set(ylabel=(f"{chain_R[j]}, ax 2 vs 0"))
    
# Draw Euler angles for left chain
fig, axes = plt.subplots(len(chain_L), 3, figsize=(8, 16), constrained_layout=True, num='left_chain_angles')
fig.suptitle('Left Chain Euler Angles', fontsize=14)
for j in range(len(chain_L)):
    eul_to_draw = eul_left[:, j] * 180/np.pi
    # round to 2 decimal points
    eul_to_draw = np.around(eul_to_draw, 2)
    axes[j,0].scatter(eul_to_draw[:,0], eul_to_draw[:,1], label=f"{chain_L[j]}, 0, 1", marker='o', s=20, color='black')
    axes[j,1].scatter(eul_to_draw[:,1], eul_to_draw[:,2], label=f"{chain_L[j]}, 1, 2", marker='o', s=20, color='black')
    axes[j,2].scatter(eul_to_draw[:,2], eul_to_draw[:,0], label=f"{chain_L[j]}, 0, 2", marker='o', s=20, color='black')
    axes[j,0].set(ylabel=(f"{chain_L[j]}, ax 0 vs 1"))
    axes[j,1].set(ylabel=(f"{chain_L[j]}, ax 1 vs 2"))
    axes[j,2].set(ylabel=(f"{chain_L[j]}, ax 0 vs 2"))

fig2, axes2 = plt.subplots(2, 3, figsize=(10, 6), constrained_layout=True, num='right_arm_angles')
fig2.suptitle('Right Upper Limb Euler Angles', fontsize=14, y=0.96)
fig2.subplots_adjust(wspace=0.5, hspace=0.4)

# axes2[0,0].scatter(sh_rot[:len(time_cl)], sh_abd[:len(time_cl)], marker='o', s=10, color='black')
# axes2[0,0].scatter(sh_rot[len(time_cl):], sh_abd[len(time_cl):], marker='o', s=10, color='blue')
axes2[0,0].scatter(sh_rot[:], sh_abd[:], marker='o', s=10, color='blue')
axes2[0,0].set(xlabel='Shoulder Rotation (deg)', ylabel='Shoulder Abduction (deg)')
axes2[0,1].scatter(sh_rot[:], sh_ext[:], marker='o', s=10, color='blue')
axes2[0,1].set(xlabel='Shoulder Rotation (deg)', ylabel='Shoulder Extension (deg)')
axes2[0,2].scatter(sh_abd[:], sh_ext[:], marker='o', s=10, color='blue')
axes2[0,2].set(xlabel='Shoulder Abduction (deg)', ylabel='Shoulder Extension (deg)')

axes2[1,0].scatter(el_fle[:], sh_rot[:], marker='o', s=10, color='blue')
axes2[1,0].set(xlabel='Elbow Flexion (deg)', ylabel='Shoulder Rotation (deg)')
axes2[1,1].scatter(el_fle[:], sh_abd[:], marker='o', s=10, color='blue')
axes2[1,1].set(xlabel='Elbow Flexion (deg)', ylabel='Shoulder Abduction (deg)')
axes2[1,2].scatter(el_fle[:], sh_ext[:], marker='o', s=10, color='blue')
axes2[1,2].set(xlabel='Elbow Flexion (deg)', ylabel='Shoulder Extension (deg)')


In [ ]:
#%% Visualization of joint locations at time t
from matplotlib.lines import Line2D

t = 300
eul_seq = 'rzxy'

def get_positions(skl, chain):  
    positions = []
    for j in range(len(chain)):
        pos = skl[(chain[j], 'Position')].iloc[t]
        positions.append(np.array(pos))
        # pos_x = skl[(chain[j], 'Position', 'X')].iloc[t]
        # pos_y = skl[(chain[j], 'Position', 'Y')].iloc[t]
        # pos_z = skl[(chain[j], 'Position', 'Z')].iloc[t]
        # positions.append(np.array([pos_x, pos_y, pos_z]))
    return positions - positions[0]

def get_euler(skl, chain):  
    eul = []
    # rot = np.zeros((len(chain),3, 3))
    R_prox = np.eye(3)
    for j in range(len(chain)):
        q = skl[(chain[j], 'Rotation')].iloc[t]
        # qx = skl[(chain[j], 'Rotation', 'X')].iloc[t]
        # qy = skl[(chain[j], 'Rotation', 'Y')].iloc[t]
        # qz = skl[(chain[j], 'Rotation', 'Z')].iloc[t]
        # qw = skl[(chain[j], 'Rotation', 'W')].iloc[t]
        # q = np.array([qx, qy, qz, qw])
        R_global = quaternion_matrix(q)[:3,:3]
        R_rel = R_prox.T @ R_global
        R_prox = R_global
        eul_values = euler_from_matrix(R_rel, axes=eul_seq)
        eul.append(eul_values)
        # if j==0:
        #     rot[j] = quaternion_matrix(q)[:3, :3]
        # else:
        #     rot[j] = rot[j-1].T @ quaternion_matrix(q)[:3, :3] 
        # eul_values = euler_from_matrix(rot[j], axes=eul_seq) 
        # eul.append(eul_values)
    return np.array(eul)

def forward_kinematics(skl, chain): 
    # eul_angles = skl_R.xs('eul', level=1, axis=1).iloc[t].values.reshape(-1, 3) * np.pi/180
    bone_pos = get_positions(skl, chain)
    bone_length = np.linalg.norm(np.diff(bone_pos, axis=0), axis=1)
    eul_angles = get_euler(skl, chain)
    # print("chain: ", chain, "\neul_angles: ", np.around(eul_angles*180/np.pi, 2))
    
    fk_pose = np.zeros((len(eul_angles), 3))
    current_position = np.zeros(3)
    current_rotation = np.eye(3) #euler_matrix(eul_angles[0, 0], eul_angles[0, 1], eul_angles[0, 2], axes=eul_seq)[:3, :3]
    fk_pose[0] = current_position
    for i in range(2):
        R = euler_matrix(eul_angles[i, 0], eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
        current_rotation = current_rotation @ R
        current_position += current_rotation @ np.array([0, 0, bone_length[i]])
        fk_pose[i+1] = current_position

    for i in range(2, len(eul_angles)-1):
        if chain==chain_R:  
        # for i in range(len(eul_angles)-1):
            if i==2:
                R = euler_matrix(eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
                current_rotation = current_rotation @ R
                current_position += current_rotation @ np.array([0, 0, bone_length[i]])
            else:
                R = euler_matrix(eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
                current_rotation = current_rotation @ R
                current_position += current_rotation @ np.array([0, bone_length[i], 0])
            fk_pose[i+1] = current_position
        # return fk_pose
        elif chain==chain_L:
        # for i in range(len(eul_angles)-1):
            R = euler_matrix(-eul_angles[i, 0], -eul_angles[i, 1], eul_angles[i, 2], axes=eul_seq)[:3, :3]
            current_rotation = current_rotation @ R
            if i==2:
                current_position += current_rotation @ np.array([0, 0, bone_length[i]])
            else:
                # current_position += current_rotation @ np.array([0, 0, bone_length[i]])
                current_position += current_rotation @ np.array([0, -bone_length[i], 0])
            fk_pose[i+1] = current_position
    return fk_pose

# Get bone positions from raw data
pos_raw_R = get_positions(skl_R, chain_R)
pos_raw_L = get_positions(skl_L, chain_L)

# Get bone positions using forward kinematics
pos_fk_R = forward_kinematics(skl_R, chain_R)
pos_fk_L = forward_kinematics(skl_L, chain_L)
print("pos_raw_L: \n", pos_raw_L[:4])
print("pos_fk_L: \n", pos_fk_L[:4])
print("pos_raw_R: \n", pos_raw_R[:4])
print("pos_fk_R: \n", pos_fk_R[:4])

# eul_angles = get_euler(skl_L, chain_L)
# print("chain: ", chain_L, "\neul_angles: ", np.around(eul_angles*180/np.pi, 2))

# Plot positions using raw position data and forward kinematics
fig, axes = plt.subplots(1, 2, subplot_kw={'projection': '3d'}, figsize=(11, 4.5), constrained_layout=True, num='Joint Locations')
# fig.subplots_adjust(left=-0.5, wspace=0.08, right=1.2)
for ax in axes:
    ax.set(xlabel='X', ylabel='Y', zlabel='Z', xlim=(-500, 500), ylim=(-500, 500), zlim=(-100, 900))
    ax.view_init(elev=25, azim=-40)
ax1, ax2 = axes[0], axes[1]
ax1.set_title('Joint Locations Using Position Data', pad=8)
ax2.set_title('Joint Locations Using Forward Kinematics', pad=8)
# colors = ['k', 'y', 'm', 'c', 'r', 'g', 'b']
colors_R = ['black', 'orange', 'navy', 'cyan', 'red', 'green', 'blue']
colors_L = ['black', 'orange', 'navy', 'purple', 'magenta', 'lime', 'teal']
for j in range(len(chain_R)):
    ax1.scatter(pos_raw_R[j][0], pos_raw_R[j][1], pos_raw_R[j][2], label=f"{chain_R[j]}, Raw", marker='*', s=50, color=colors_R[j])
    ax1.scatter(pos_raw_L[j][0], pos_raw_L[j][1], pos_raw_L[j][2], label=f"{chain_L[j]}, Raw", marker='*', s=50, color=colors_L[j])
    ax2.scatter(pos_fk_R[j][0], pos_fk_R[j][1], pos_fk_R[j][2], label=f"{chain_R[j]}, FK", marker='o', s=50, color=colors_R[j])
    ax2.scatter(pos_fk_L[j][0], pos_fk_L[j][1], pos_fk_L[j][2], label=f"{chain_L[j]}, FK", marker='o', s=50, color=colors_L[j])
    if j!=0:
        ax1.plot([pos_raw_R[j][0], pos_raw_R[j-1][0]], [pos_raw_R[j][1], pos_raw_R[j-1][1]], [pos_raw_R[j][2], pos_raw_R[j-1][2]], 'k')
        ax1.plot([pos_raw_L[j][0], pos_raw_L[j-1][0]], [pos_raw_L[j][1], pos_raw_L[j-1][1]], [pos_raw_L[j][2], pos_raw_L[j-1][2]], 'b')
        ax2.plot([pos_fk_R[j][0], pos_fk_R[j-1][0]], [pos_fk_R[j][1], pos_fk_R[j-1][1]], [pos_fk_R[j][2], pos_fk_R[j-1][2]], 'k')
        ax2.plot([pos_fk_L[j][0], pos_fk_L[j-1][0]], [pos_fk_L[j][1], pos_fk_L[j-1][1]], [pos_fk_L[j][2], pos_fk_L[j-1][2]], 'b')
ax2.legend(handles=
    [Line2D([],[], color=colors_R[j], linestyle='', marker='o', markersize=8, label=f"{chain_R[j]}") for j in range(len(chain_R))] + 
    [Line2D([],[], color=colors_L[j], linestyle='', marker='o', markersize=8, label=f"{chain_L[j]}") for j in range(3, len(chain_L))] + 
    [Line2D([],[], color='black', linestyle='-', label='Bones')],  loc='right', bbox_to_anchor=(1.4, 0.5))
plt.show()
